In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
singleword=[]
for key,val in senticnet.items():
    if(len(key.split('_'))==1):
        singleword.append(key)
print(len(singleword))
word=[]
primary=[]
sec=[]
pola=[]
for x in singleword:
    word.append(x)
    primary.append(senticnet[x][4])
    sec.append(senticnet[x][5])
    pola.append(senticnet[x][7])
df_emo=pd.DataFrame(list(zip(word,primary,sec,pola)),columns=["Word","Primary","Secondary","Polarity"])
df_emo.sample(10)

39891


,Word,Primary,Secondary,Polarity
25030,overcome,#joy,#admiration,0.881
37463,unmarried,#sadness,#disgust,-0.81
14788,goldfish,#sadness,#anger,-0.78
9387,dictatorship,#fear,#disgust,-0.75
12383,extension,#interest,#admiration,0.161
18492,intimidation,#anger,#disgust,-0.87
23129,myotonia,#anger,#disgust,-0.68
31560,senile,#sadness,#fear,-0.87
19134,judicious,#surprise,#admiration,0.762
36829,unapproachable,#sadness,#disgust,-0.94


In [4]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from skmultilearn.ensemble import RakelD

In [5]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
X = df['Text']
Y = df[['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']]

In [6]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Text'])
print(len(vectorizer.vocabulary_))

17402


In [7]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [8]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [10]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    classifier = RakelD(base_classifier = RandomForestClassifier(),
                        base_classifier_require_dense=[False, False],
                        labelset_size=4)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    
    y_pred_val = y_pred.toarray()
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)
Hamming Loss: 0.16304347826086957
Exact Prediction: 0.23971797884841364
Macro F-Score: 0.8846109921881393
Micro F-Score: 0.9051282051282051
Average Accuracy: 0.8369565217391305


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.1686764705882353
Exact Prediction: 0.23411764705882354
Macro F-Score: 0.8850478335708134
Micro F-Score: 0.9016210652714641
Average Accuracy: 0.8313235294117647


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.1698529411764706
Exact Prediction: 0.20352941176470588
Macro F-Score: 0.880914505296437
Micro F-Score: 0.8999740192257729
Average Accuracy: 0.8301470588235293


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.1673529411764706
Exact Prediction: 0.2576470588235294
Macro F-Score: 0.8673787748501733
Micro F-Score: 0.9004722756690572
Average Accuracy: 0.8326470588235294


k_fold validation: 5


In [11]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))


In [12]:
df_rfc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_rfc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.163043,0.239718,0.884611,0.905128,0.836957
1,2,0.168676,0.234118,0.885048,0.901621,0.831324
2,3,0.169853,0.203529,0.880915,0.899974,0.830147
3,4,0.167353,0.257647,0.867379,0.900472,0.832647
4,5,0.169706,0.241176,0.878691,0.900517,0.830294
5,6,0.171471,0.250588,0.869171,0.898855,0.828529
6,7,0.170735,0.263529,0.868029,0.898345,0.829265
7,8,0.169265,0.270588,0.880496,0.900665,0.830735
8,9,0.168235,0.214118,0.889467,0.902985,0.831765
9,10,0.168824,0.228235,0.882013,0.900915,0.831176


In [13]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    classifier = RakelD(base_classifier = SVC(),
                        base_classifier_require_dense=[False, False],
                        labelset_size=4)
    classifier.fit(x_train,y_train)
    y_pred = classifier.predict(x_test)
    
    y_pred_val = y_pred.toarray()
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val.T.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)
Hamming Loss: 0.1605464159811986
Exact Prediction: 0.2291421856639248
Macro F-Score: 0.8855431645248196
Micro F-Score: 0.9062848323758896
Average Accuracy: 0.8394535840188014


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.17
Exact Prediction: 0.18352941176470589
Macro F-Score: 0.8906761367523182
Micro F-Score: 0.9026772183869338
Average Accuracy: 0.83


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.1711764705882353
Exact Prediction: 0.22588235294117648
Macro F-Score: 0.8624944774127281
Micro F-Score: 0.8978230337078651
Average Accuracy: 0.8288235294117647


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
Hamming Loss: 0.1698529411764706
Exact Prediction: 0.26235294117647057
Macro F-Score: 0.8612303301039919
Micro F-Score: 0.8991002009260067
Average Accuracy: 0.8301470588235293


k_fold validation: 5
(7651, 17402) (850, 17402)
(

In [14]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [15]:
df_svc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_svc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.160546,0.229142,0.885543,0.906285,0.839454
1,2,0.170000,0.183529,0.890676,0.902677,0.830000
2,3,0.171176,0.225882,0.862494,0.897823,0.828824
3,4,0.169853,0.262353,0.861230,0.899100,0.830147
4,5,0.175735,0.243529,0.855692,0.895752,0.824265
5,6,0.161029,0.189412,0.897361,0.908191,0.838971
6,7,0.164412,0.232941,0.878753,0.902935,0.835588
7,8,0.171765,0.277647,0.860788,0.898116,0.828235
8,9,0.167353,0.201176,0.891022,0.903788,0.832647
9,10,0.167794,0.240000,0.873989,0.901101,0.832206
